In [9]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score

# Load the dataset
train_data = pd.read_csv("dataset/train.csv")
test_data = pd.read_csv("dataset/test.csv")

# Data Preprocessing
features = ["Pclass", "Sex", "Age", "SibSp", "Parch"]
target = "Survived"

def preprocess_data(df):
    df["Age"].fillna(df["Age"].mean(), inplace=True)
    df["Sex"] = df["Sex"].map({"male": 0, "female": 1})
    df["FamilySize"] = df["SibSp"] + df["Parch"]
    return df[features]

X_train = preprocess_data(train_data)
y_train = train_data[target]
X_test = preprocess_data(test_data)

# Apply SMOTE to balance the data
smote = SMOTE(random_state=1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Split the resampled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=1)

# Define a Keras neural network model
def create_keras_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation="relu"))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

keras_classifier = KerasClassifier(build_fn=create_keras_model, epochs=60, batch_size=64, verbose=1)

# Ensemble Learning (Soft Voting) including Keras NN
models = [
    ("Random Forest", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)),
    ("Gradient Boosting", GradientBoostingClassifier(n_estimators=100, random_state=1)),
    ("Decision Tree", DecisionTreeClassifier(criterion="entropy", max_depth=4, random_state=1)),
    ("SVM (RBF Kernel)", SVC(kernel="rbf", C=1.0, gamma=0.2, probability=True, random_state=1)),
    ("Logistic Regression", LogisticRegression(random_state=1)),
    ("Keras Neural Network", keras_classifier),
]

ensemble = VotingClassifier(models, voting="soft")

# Train the ensemble model
ensemble.fit(X_train, y_train)

# Evaluate the ensemble model on the validation set
ensemble_accuracy = accuracy_score(y_val, ensemble.predict(X_val))
print(f"Ensemble Model Accuracy: {ensemble_accuracy}")

# Get probability estimates for test data
test_probabilities = ensemble.predict_proba(X_test)[:, 1]

# Apply a threshold to convert probabilities to binary predictions
print(test_probabilities)
threshold = 0.7
test_predictions = (test_probabilities > threshold).astype(int)

# Save the test predictions to a CSV file
output = pd.DataFrame({"PassengerId": test_data["PassengerId"], "Survived": test_predictions})
output.to_csv("submission_ensemble_with_keras.csv", index=False)
print("Test predictions saved to submission_ensemble_with_keras.csv")


Epoch 1/60


d:\Python39\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


14/14 [==============================] - 1s 2ms/step - loss: 0.6893 - accuracy: 0.5421
Epoch 2/60
14/14 [==============================] - 0s 2ms/step - loss: 0.6542 - accuracy: 0.6731
Epoch 3/60
14/14 [==============================] - 0s 2ms/step - loss: 0.6413 - accuracy: 0.6606
Epoch 4/60
14/14 [==============================] - 0s 2ms/step - loss: 0.6030 - accuracy: 0.7677
Epoch 5/60
14/14 [==============================] - 0s 2ms/step - loss: 0.5804 - accuracy: 0.7585
Epoch 6/60
14/14 [==============================] - 0s 2ms/step - loss: 0.5496 - accuracy: 0.7688
Epoch 7/60
14/14 [==============================] - 0s 2ms/step - loss: 0.5169 - accuracy: 0.7779
Epoch 8/60
14/14 [==============================] - 0s 2ms/step - loss: 0.4961 - accuracy: 0.7711
Epoch 9/60
14/14 [==============================] - 0s 2ms/step - loss: 0.4916 - accuracy: 0.7756
Epoch 10/60
14/14 [==============================] - 0s 2ms/step - loss: 0.4820 - accuracy: 0.7882
Epoch 11/60
14/14 [===========